# MOVIES PROJECT

necesary imports

In [6]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

framing function

In [7]:
def frame_it(list_of_pd_series):
    return pd.concat(list_of_pd_series, axis=1)

urls into responses

In [8]:
response_1_50 = requests.get('https://www.imdb.com/search/title/?groups=top_100&sort=user_rating,asc')
response_51_100 = requests.get('https://www.imdb.com/search/title/?groups=top_100&sort=user_rating,asc&start=51&ref_=adv_nxt')

soupify the responses

In [9]:
soup_1_50 = BeautifulSoup(response_1_50.text,'html.parser')
soup_51_100 = BeautifulSoup(response_51_100.text,'html.parser')

## RELEASE DATES

traverse the soup in search for the dates

In [10]:
movie_release_dates_1_50 = soup_1_50.findAll('span',class_ = 'lister-item-year text-muted unbold')
movie_release_dates_51_100 = soup_51_100.findAll('span',class_ = 'lister-item-year text-muted unbold')

movie_release_dates = movie_release_dates_1_50 + movie_release_dates_51_100

date cleaning function

In [11]:
def clean_date(date_string):

    # fix stupid strings
    date_string = (date_string.text).replace("(", "")
    date_string = date_string.replace(")", "")
    date_string = date_string.replace("I","")
    date_string = date_string.strip()

    # change them into integers
    date_int = int(date_string)

    return date_int

create an empty list for release dates

In [12]:
# list_rel_dates_1_50 = []
# list_rel_dates51_100 = []

list_rel_dates = []

process dates

In [13]:
for str_date in movie_release_dates:
    list_rel_dates.append( clean_date(str_date) )

make them into pd.Series

In [14]:
rel_dates_series = pd.Series(list_rel_dates, name="Release Date")

## RUNTIMES

traverse the soup in search for runtimes

In [15]:
runtimes_1_50 = soup_1_50.findAll('span',class_ = "runtime")
runtimes_51_100 = soup_51_100.findAll('span',class_ = "runtime")

runtimes = runtimes_1_50 + runtimes_51_100

create empty lists for runtimes

In [16]:
movie_runtimes = []

create a time-cleaning function

In [17]:
def clean_time(time_string):

    time_string = time_string.split()
    time_string = time_string[0]

    return int(time_string)

process runtimes

In [18]:
for runtime in runtimes:
    movie_runtimes.append( clean_time(runtime.text) )

make them into pd.Series

In [19]:
runtimes_series = pd.Series(movie_runtimes, name="Runtime")

## MOVIE NAMES

traverse the soup

In [20]:
movie_names_1_50 = soup_1_50.find_all(name="h3", class_="lister-item-header")
movie_names_51_100 = soup_51_100.find_all(name="h3", class_="lister-item-header")

movie_names = movie_names_1_50 + movie_names_51_100

create an empty list for movie names

In [21]:
names_list = []

process names

In [22]:
for mn in movie_names:
    names_list.append(list(mn)[3].text)

make them into pd.Series

In [23]:
names_series = pd.Series(names_list, name="Movie name")

## MOVIE TYPES

traverse the soup

In [24]:
genres_1_50 = soup_1_50.find_all(name="span", class_="genre")
genres_51_100 = soup_51_100.find_all(name="span", class_="genre")

genres = genres_1_50 + genres_51_100

create an empty list, process and serialise it

In [25]:
genres_list = []
for gen in genres:
    string = list(gen)[0].replace("\n", "").strip()
    genres_list.append(string)
genres_series = pd.Series(genres_list, name="Genre")

## RATING

get stuff from the soup and serialise it

In [211]:
# <div class="inline-block ratings-imdb-rating" name="ir" data-value="8,3">
#     <span class="global-sprite rating-star imdb-rating"></span>
#     <strong>8,3</strong>
# </div>

In [26]:
ratings = soup_1_50.find_all(name="div", attrs={'name':'ir'}) + soup_51_100.find_all(name="div", attrs={'name':'ir'})
ratings_series = pd.Series([ float(rt.text.strip()) for rt in ratings ], name="Rating")

## PROFIT!

In [29]:
all_series = [names_series, ratings_series, rel_dates_series, runtimes_series, genres_series]
# df = frame_it(all_series)


# %timeit df = pd.DataFrame([all_series, runtimes_series]).T # this is basicaly shit.
df = pd.concat(all_series, axis=1)

action_df = df.loc[df['Genre'].str.contains('Action')]
action_df

,Movie name,Rating,Release Date,Runtime,Genre
7,Gwiezdne wojny: Część VI - Powrót Jedi,8.3,1983,131,"Action, Adventure, Fantasy"
9,Obcy - decydujące starcie,8.3,1986,137,"Action, Adventure, Sci-Fi"
29,Poszukiwacze zaginionej arki,8.4,1981,115,"Action, Adventure"
31,Księżniczka Mononoke,8.4,1997,134,"Animation, Action, Adventure"
33,Oldboy,8.4,2003,120,"Action, Drama, Mystery"
38,Mroczny Rycerz powstaje,8.4,2012,164,"Action, Adventure"
41,Avengers: Wojna bez granic,8.4,2018,149,"Action, Adventure, Sci-Fi"
42,Avengers: Koniec gry,8.4,2019,181,"Action, Adventure, Drama"
43,Spider-Man Uniwersum,8.4,2018,117,"Animation, Action, Adventure"
56,Terminator 2: Dzień sądu,8.5,1991,137,"Action, Sci-Fi"
